In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import nltk
import numpy as np
from IPython.display import clear_output
import json
from sklearn.ensemble import RandomForestRegressor
from xgboost.sklearn import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
%matplotlib inline
nltk.download('stopwords')

Считывание из файла

In [19]:
with open('data10000.json', 'r') as file:
    data2 = json.load(file)

data2['colour'] = list(map(lambda x: [0, 0, 0] if x == None else x, data2['colour']))
data2['text'] = list(map(lambda x: ' '.join(x), data2['text']))

In [ ]:
train_index = []
test_index = []
loaded_data = pd.DataFrame(data2)
colours = pd.DataFrame(list(loaded_data['colour']), columns=["red", "green", 'blue'])
vectorizer = CountVectorizer(ngram_range=(1,1))
vectorized_texts = vectorizer.fit_transform(data2['text'])
vocabulary = list(map(lambda x: 'word_' + x, vectorizer.get_feature_names_out()))

for i in loaded_data.index:
    result = np.random.choice(2, p=[.65,.35])
    if result == 1:
        test_index.append(i)
    else:
        train_index.append(i)

y_train = loaded_data.iloc[train_index]['virality'].astype(int)
x_train = loaded_data.iloc[train_index][['photo', 'video', 'audio', 'words']]
x_train = pd.merge(x_train, pd.DataFrame(vectorized_texts[train_index].toarray(), index=x_train.index, columns=vocabulary), left_index=True, right_index=True)
x_train = pd.merge(x_train, colours, left_index=True, right_index=True)

y_test = loaded_data.iloc[test_index]['virality'].astype(int)
x_test = loaded_data.iloc[test_index][['photo', 'video', 'audio', 'words']]
x_test = pd.merge(x_test, pd.DataFrame(vectorized_texts[test_index].toarray(), index=x_test.index, columns=vocabulary), left_index=True, right_index=True)
x_test = pd.merge(x_test, colours, left_index=True, right_index=True)

x_train
y_actual = y_test

In [ ]:
x_train

Random forest

In [ ]:
#rf = RandomForestRegressor(n_estimators=1000, verbose=True)
#rf.fit(x_train, y_train)

In [ ]:
#rf_y_pred = rf.predict(x_test)

#rf_deltas = pd.DataFrame(list(zip(rf_y_pred, y_actual, (rf_y_pred -
#y_actual)/(y_actual))), columns=['predicted', 'actual', 'delta'])

XGBoost

In [ ]:
#xgb = XGBRegressor()
#xgb.fit(x_train, y_train)

In [ ]:
#xgb_y_pred = xgb.predict(x_test)

#xgb_deltas = pd.DataFrame(list(zip(xgb_y_pred, y_actual, (xgb_y_pred -
#y_actual)/(y_actual))), columns=['predicted', 'actual', 'delta'])

CatBoost

In [ ]:
cb = CatBoostRegressor()
cb.fit(x_train, y_train, verbose=False)

In [ ]:
cb_y_pred = cb.predict(x_test)

cb_deltas = pd.DataFrame(list(zip(cb_y_pred, y_actual, (cb_y_pred -
y_actual)/(y_actual))), columns=['predicted', 'actual', 'delta'])


SVR

In [ ]:
#svr = SVR()
#svr.fit(x_train, y_train)

In [ ]:
#svr_y_pred = svr.predict(x_test)

#svr_deltas = pd.DataFrame(list(zip(svr_y_pred, y_actual, (svr_y_pred -
#y_actual)/(y_actual))), columns=['predicted', 'actual', 'delta'])

MPL

In [ ]:
#mpl = MLPRegressor()
#mpl.fit(x_train, y_train)

In [ ]:
#mpl_y_pred = mpl.predict(x_test)

#mpl_deltas = pd.DataFrame(list(zip(mpl_y_pred, y_actual, (mpl_y_pred -
#y_actual)/(y_actual))), columns=['predicted', 'actual', 'delta'])

Вывод

In [ ]:
#print(f"Random forest = {np.sqrt(np.mean((rf_y_pred-y_actual)**2))/np.mean(y_actual)}")
#print(f"XGBoost = {np.sqrt(np.mean((xgb_y_pred-y_actual)**2))/np.mean(y_actual)}")
print(f"CatBoost = {np.sqrt(np.mean((cb_y_pred-y_actual)**2))/np.mean(y_actual)}")
#print(f"SVR = {np.sqrt(np.mean((svr_y_pred-y_actual)**2))/np.mean(y_actual)}")
#print(f"MPL = {np.sqrt(np.mean((mpl_y_pred-y_actual)**2))/np.mean(y_actual)}")

График

In [ ]:
cb_deltas[['predicted','actual']].iloc[:30,:].plot(kind='bar', figsize=(16,8))

Выгрузка

In [ ]:
cb.save_model("model")